In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv('./movie_meta.csv', engine='python', encoding='CP949')
print(movies.shape)
movies.head()

(1668, 40)


,movie_id,title,release_year,release_date,runtime,mpa_rating,mpa_rating_origin,imdb_score,votes,reviews_users,...,contract_price,studio_score,price_class,writer_1,writer_2,actor_1,actor_2,actor_3,description,storyline
0,tt0472033,9,2009,09-09-2009,79,PG-13,PG-13,7.1,127563,267,...,2195,6,1,Pamela Pettler,Shane Acker,Christopher Plummer,Martin Landau,John C. Reilly,A rag doll that awakens in a postapocalyptic f...,In a world destroyed in a war between man and ...
1,tt0478087,21,2008,03-28-2008,123,PG-13,PG-13,6.8,216963,302,...,2295,10,1,Peter Steinfeld,Allan Loeb,Jim Sturgess,Kevin Spacey,Kate Bosworth,"""21"" is the fact-based story about six MIT stu...","Ben Campbell is a young, highly intelligent, s..."
2,tt0453562,42,2013,04-12-2013,128,PG-13,PG-13,7.5,79204,234,...,2700,10,10,Brian Helgeland,NaN,Chadwick Boseman,Harrison Ford,Nicole Beharie,"In 1947, Jackie Robinson becomes the first Afr...","In 1946, Jackie Robinson is a Negro League bas..."
3,tt0416449,300,2006,03-09-2007,117,R,R,7.7,702777,2142,...,2700,10,10,Zack Snyder,Kurt Johnstad,Gerard Butler,Lena Headey,Dominic West,King Leonidas of Sparta and a force of 300 men...,In the Battle of Thermopylae of 480 BC an alli...
4,tt0450385,1408,2007,06-22-2007,104,PG-13,PG-13,6.8,245370,622,...,2145,4,1,Matt Greenberg,Scott Alexander,John Cusack,Tony Shalhoub,Len Cariou,A man who specialises in debunking paranormal ...,The cynical and skeptical writer Mike Enslin w...


In [3]:
movies.shape

(1668, 40)

In [4]:
tmp_m = movies['reviews_users'].quantile(0.89)
tmp_m

895.4100000000008

In [5]:
tmp_data = movies.copy().loc[movies['votes']>=tmp_m]
tmp_data.shape

(1668, 40)

In [6]:
del tmp_data

m = movies['reviews_users'].quantile(0.9)
movies = movies.loc[movies['votes']>=m]

In [7]:
movies.head()

,movie_id,title,release_year,release_date,runtime,mpa_rating,mpa_rating_origin,imdb_score,votes,reviews_users,...,contract_price,studio_score,price_class,writer_1,writer_2,actor_1,actor_2,actor_3,description,storyline
0,tt0472033,9,2009,09-09-2009,79,PG-13,PG-13,7.1,127563,267,...,2195,6,1,Pamela Pettler,Shane Acker,Christopher Plummer,Martin Landau,John C. Reilly,A rag doll that awakens in a postapocalyptic f...,In a world destroyed in a war between man and ...
1,tt0478087,21,2008,03-28-2008,123,PG-13,PG-13,6.8,216963,302,...,2295,10,1,Peter Steinfeld,Allan Loeb,Jim Sturgess,Kevin Spacey,Kate Bosworth,"""21"" is the fact-based story about six MIT stu...","Ben Campbell is a young, highly intelligent, s..."
2,tt0453562,42,2013,04-12-2013,128,PG-13,PG-13,7.5,79204,234,...,2700,10,10,Brian Helgeland,NaN,Chadwick Boseman,Harrison Ford,Nicole Beharie,"In 1947, Jackie Robinson becomes the first Afr...","In 1946, Jackie Robinson is a Negro League bas..."
3,tt0416449,300,2006,03-09-2007,117,R,R,7.7,702777,2142,...,2700,10,10,Zack Snyder,Kurt Johnstad,Gerard Butler,Lena Headey,Dominic West,King Leonidas of Sparta and a force of 300 men...,In the Battle of Thermopylae of 480 BC an alli...
4,tt0450385,1408,2007,06-22-2007,104,PG-13,PG-13,6.8,245370,622,...,2145,4,1,Matt Greenberg,Scott Alexander,John Cusack,Tony Shalhoub,Len Cariou,A man who specialises in debunking paranormal ...,The cynical and skeptical writer Mike Enslin w...


In [8]:
c = movies['imdb_score'].mean()

In [9]:
print(m)
print(c)

941.4999999999998
6.547661870503608


In [10]:
def weighted_rating(x, m=m, c=c):
    v = x['votes']
    R = x['imdb_score']
    
    return (v / (v+m) * R) + (m / (m+v) * c)

In [11]:
movies['score'] = movies.apply(weighted_rating, axis=1)

In [12]:
movies['genres'] = movies['Genre_1']+str(' ')+movies['Genre_2']+str(' ')+movies['Genre_3']
movies['genres']

0       Animation Action Adventure
1              Crime Drama History
2            Biography Drama Sport
3                              NaN
4           Fantasy Horror Mystery
                   ...            
1663           Crime Drama Mystery
1664      Adventure Comedy Fantasy
1665         Comedy Family Fantasy
1666       Action Adventure Comedy
1667    Animation Adventure Comedy
Name: genres, Length: 1668, dtype: object

In [13]:
movies.drop("Genre_1", axis=1, inplace=True)
movies

,movie_id,title,release_year,release_date,runtime,mpa_rating,mpa_rating_origin,imdb_score,votes,reviews_users,...,price_class,writer_1,writer_2,actor_1,actor_2,actor_3,description,storyline,score,genres
0,tt0472033,9,2009,09-09-2009,79,PG-13,PG-13,7.1,127563,267,...,1,Pamela Pettler,Shane Acker,Christopher Plummer,Martin Landau,John C. Reilly,A rag doll that awakens in a postapocalyptic f...,In a world destroyed in a war between man and ...,7.095953,Animation Action Adventure
1,tt0478087,21,2008,03-28-2008,123,PG-13,PG-13,6.8,216963,302,...,1,Peter Steinfeld,Allan Loeb,Jim Sturgess,Kevin Spacey,Kate Bosworth,"""21"" is the fact-based story about six MIT stu...","Ben Campbell is a young, highly intelligent, s...",6.798910,Crime Drama History
2,tt0453562,42,2013,04-12-2013,128,PG-13,PG-13,7.5,79204,234,...,10,Brian Helgeland,NaN,Chadwick Boseman,Harrison Ford,Nicole Beharie,"In 1947, Jackie Robinson becomes the first Afr...","In 1946, Jackie Robinson is a Negro League bas...",7.488813,Biography Drama Sport
3,tt0416449,300,2006,03-09-2007,117,R,R,7.7,702777,2142,...,10,Zack Snyder,Kurt Johnstad,Gerard Butler,Lena Headey,Dominic West,King Leonidas of Sparta and a force of 300 men...,In the Battle of Thermopylae of 480 BC an alli...,7.698458,NaN
4,tt0450385,1408,2007,06-22-2007,104,PG-13,PG-13,6.8,245370,622,...,1,Matt Greenberg,Scott Alexander,John Cusack,Tony Shalhoub,Len Cariou,A man who specialises in debunking paranormal ...,The cynical and skeptical writer Mike Enslin w...,6.799035,Fantasy Horror Mystery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,tt0443706,Zodiac,2007,03-02-2007,157,R,R,7.7,407046,698,...,1,James Vanderbilt,Robert Graysmith,Jake Gyllenhaal,Mark Ruffalo,Anthony Edwards,"In the late 1960s/early 1970s, a San Francisco...",A serial killer in the San Francisco Bay Area ...,7.697341,Crime Drama Mystery
1664,tt1156398,Zombieland,2009,10-02-2009,88,R,R,7.6,482780,680,...,10,Rhett Reese,Paul Wernick,Jesse Eisenberg,Woody Harrelson,Emma Stone,A shy student trying to reach his family in Oh...,Searching for family. In the early twenty-firs...,7.597952,Adventure Comedy Fantasy
1665,tt1222817,Zookeeper,2011,07-08-2011,102,PG,PG,5.2,53731,132,...,1,Nick Bakay,Rock Reuben,Kevin James,Rosario Dawson,Leslie Bibb,A group of zoo animals decide to break their c...,A group of zoo animals decide to break their c...,5.223208,Comedy Family Fantasy
1666,tt1608290,Zoolander 2,2016,02-12-2016,101,PG-13,PG-13,4.7,61163,213,...,1,Justin Theroux,Ben Stiller,Justin Bieber,Jon Daly,Pen lope Cruz,Derek and Hansel are lured into modelling agai...,After many years of being separated from model...,4.728010,Action Adventure Comedy


In [14]:
movies.drop("Genre_2", axis=1, inplace=True)
movies

,movie_id,title,release_year,release_date,runtime,mpa_rating,mpa_rating_origin,imdb_score,votes,reviews_users,...,price_class,writer_1,writer_2,actor_1,actor_2,actor_3,description,storyline,score,genres
0,tt0472033,9,2009,09-09-2009,79,PG-13,PG-13,7.1,127563,267,...,1,Pamela Pettler,Shane Acker,Christopher Plummer,Martin Landau,John C. Reilly,A rag doll that awakens in a postapocalyptic f...,In a world destroyed in a war between man and ...,7.095953,Animation Action Adventure
1,tt0478087,21,2008,03-28-2008,123,PG-13,PG-13,6.8,216963,302,...,1,Peter Steinfeld,Allan Loeb,Jim Sturgess,Kevin Spacey,Kate Bosworth,"""21"" is the fact-based story about six MIT stu...","Ben Campbell is a young, highly intelligent, s...",6.798910,Crime Drama History
2,tt0453562,42,2013,04-12-2013,128,PG-13,PG-13,7.5,79204,234,...,10,Brian Helgeland,NaN,Chadwick Boseman,Harrison Ford,Nicole Beharie,"In 1947, Jackie Robinson becomes the first Afr...","In 1946, Jackie Robinson is a Negro League bas...",7.488813,Biography Drama Sport
3,tt0416449,300,2006,03-09-2007,117,R,R,7.7,702777,2142,...,10,Zack Snyder,Kurt Johnstad,Gerard Butler,Lena Headey,Dominic West,King Leonidas of Sparta and a force of 300 men...,In the Battle of Thermopylae of 480 BC an alli...,7.698458,NaN
4,tt0450385,1408,2007,06-22-2007,104,PG-13,PG-13,6.8,245370,622,...,1,Matt Greenberg,Scott Alexander,John Cusack,Tony Shalhoub,Len Cariou,A man who specialises in debunking paranormal ...,The cynical and skeptical writer Mike Enslin w...,6.799035,Fantasy Horror Mystery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,tt0443706,Zodiac,2007,03-02-2007,157,R,R,7.7,407046,698,...,1,James Vanderbilt,Robert Graysmith,Jake Gyllenhaal,Mark Ruffalo,Anthony Edwards,"In the late 1960s/early 1970s, a San Francisco...",A serial killer in the San Francisco Bay Area ...,7.697341,Crime Drama Mystery
1664,tt1156398,Zombieland,2009,10-02-2009,88,R,R,7.6,482780,680,...,10,Rhett Reese,Paul Wernick,Jesse Eisenberg,Woody Harrelson,Emma Stone,A shy student trying to reach his family in Oh...,Searching for family. In the early twenty-firs...,7.597952,Adventure Comedy Fantasy
1665,tt1222817,Zookeeper,2011,07-08-2011,102,PG,PG,5.2,53731,132,...,1,Nick Bakay,Rock Reuben,Kevin James,Rosario Dawson,Leslie Bibb,A group of zoo animals decide to break their c...,A group of zoo animals decide to break their c...,5.223208,Comedy Family Fantasy
1666,tt1608290,Zoolander 2,2016,02-12-2016,101,PG-13,PG-13,4.7,61163,213,...,1,Justin Theroux,Ben Stiller,Justin Bieber,Jon Daly,Pen lope Cruz,Derek and Hansel are lured into modelling agai...,After many years of being separated from model...,4.728010,Action Adventure Comedy


In [15]:
movies.drop("Genre_3", axis=1, inplace=True)
movies

,movie_id,title,release_year,release_date,runtime,mpa_rating,mpa_rating_origin,imdb_score,votes,reviews_users,...,price_class,writer_1,writer_2,actor_1,actor_2,actor_3,description,storyline,score,genres
0,tt0472033,9,2009,09-09-2009,79,PG-13,PG-13,7.1,127563,267,...,1,Pamela Pettler,Shane Acker,Christopher Plummer,Martin Landau,John C. Reilly,A rag doll that awakens in a postapocalyptic f...,In a world destroyed in a war between man and ...,7.095953,Animation Action Adventure
1,tt0478087,21,2008,03-28-2008,123,PG-13,PG-13,6.8,216963,302,...,1,Peter Steinfeld,Allan Loeb,Jim Sturgess,Kevin Spacey,Kate Bosworth,"""21"" is the fact-based story about six MIT stu...","Ben Campbell is a young, highly intelligent, s...",6.798910,Crime Drama History
2,tt0453562,42,2013,04-12-2013,128,PG-13,PG-13,7.5,79204,234,...,10,Brian Helgeland,NaN,Chadwick Boseman,Harrison Ford,Nicole Beharie,"In 1947, Jackie Robinson becomes the first Afr...","In 1946, Jackie Robinson is a Negro League bas...",7.488813,Biography Drama Sport
3,tt0416449,300,2006,03-09-2007,117,R,R,7.7,702777,2142,...,10,Zack Snyder,Kurt Johnstad,Gerard Butler,Lena Headey,Dominic West,King Leonidas of Sparta and a force of 300 men...,In the Battle of Thermopylae of 480 BC an alli...,7.698458,NaN
4,tt0450385,1408,2007,06-22-2007,104,PG-13,PG-13,6.8,245370,622,...,1,Matt Greenberg,Scott Alexander,John Cusack,Tony Shalhoub,Len Cariou,A man who specialises in debunking paranormal ...,The cynical and skeptical writer Mike Enslin w...,6.799035,Fantasy Horror Mystery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,tt0443706,Zodiac,2007,03-02-2007,157,R,R,7.7,407046,698,...,1,James Vanderbilt,Robert Graysmith,Jake Gyllenhaal,Mark Ruffalo,Anthony Edwards,"In the late 1960s/early 1970s, a San Francisco...",A serial killer in the San Francisco Bay Area ...,7.697341,Crime Drama Mystery
1664,tt1156398,Zombieland,2009,10-02-2009,88,R,R,7.6,482780,680,...,10,Rhett Reese,Paul Wernick,Jesse Eisenberg,Woody Harrelson,Emma Stone,A shy student trying to reach his family in Oh...,Searching for family. In the early twenty-firs...,7.597952,Adventure Comedy Fantasy
1665,tt1222817,Zookeeper,2011,07-08-2011,102,PG,PG,5.2,53731,132,...,1,Nick Bakay,Rock Reuben,Kevin James,Rosario Dawson,Leslie Bibb,A group of zoo animals decide to break their c...,A group of zoo animals decide to break their c...,5.223208,Comedy Family Fantasy
1666,tt1608290,Zoolander 2,2016,02-12-2016,101,PG-13,PG-13,4.7,61163,213,...,1,Justin Theroux,Ben Stiller,Justin Bieber,Jon Daly,Pen lope Cruz,Derek and Hansel are lured into modelling agai...,After many years of being separated from model...,4.728010,Action Adventure Comedy


In [16]:
transformer = CountVectorizer()

In [17]:
genres_vector = transformer.fit_transform(movies['genres'].astype("U"))

In [18]:
genres_vector.shape

(1668, 22)

In [19]:
similarity = cosine_similarity(genres_vector, genres_vector)
similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.66666667,
        0.66666667],
       [0.        , 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.33333333,
        0.33333333],
       [0.66666667, 0.        , 0.        , ..., 0.33333333, 1.        ,
        0.66666667],
       [0.66666667, 0.        , 0.        , ..., 0.33333333, 0.66666667,
        1.        ]])

In [20]:
similarity = similarity.argsort()
similarity

array([[ 833,  881,  880, ..., 1517,  148,    0],
       [   0,  972,  967, ...,  310, 1212,    1],
       [   0,  993,  992, ..., 1283, 1556, 1175],
       ...,
       [   0, 1028, 1027, ...,  372,  801, 1070],
       [ 833, 1506,  820, ..., 1411, 1629, 1364],
       [ 833,  903,  901, ..., 1548, 1160, 1667]])

In [21]:
similarity = similarity[:, ::-1]
similarity

array([[   0,  148, 1517, ...,  880,  881,  833],
       [   1, 1212,  310, ...,  967,  972,    0],
       [1175, 1556, 1283, ...,  992,  993,    0],
       ...,
       [1070,  801,  372, ..., 1027, 1028,    0],
       [1364, 1629, 1411, ...,  820, 1506,  833],
       [1667, 1160, 1548, ...,  901,  903,  833]])

In [22]:
similarity.shape

(1668, 1668)

In [23]:
def get_recommend_movie_list(movies, movie_title, top=30):
    search_df = movies[movies['title'] == movie_title]
    search_df_index = search_df.index.values
    similarity_index = similarity[search_df_index, :top].reshape(-1)
    similarity_index = similarity_index[similarity_index != search_df_index]
    result = movies.iloc[similarity_index].sort_values('score', ascending=False)[:10]
    
    return result

In [26]:
get_recommend_movie_list(movies, movie_title='Avatar')

,movie_id,title,release_year,release_date,runtime,mpa_rating,mpa_rating_origin,imdb_score,votes,reviews_users,...,price_class,writer_1,writer_2,actor_1,actor_2,actor_3,description,storyline,score,genres
1090,tt0121766,Star Wars: Episode III - Revenge of the Sith,2005,05-19-2005,140,PG-13,PG-13,7.5,655343,3385,...,10,George Lucas,NaN,Ewan McGregor,Natalie Portman,Hayden Christensen,"Three years into the Clone Wars, the Jedi resc...",Nearly three years have passed since the begin...,7.498634,Action Adventure Fantasy
324,tt1211837,Doctor Strange,2016,11-04-2016,115,PG-13,PG-13,7.5,541463,880,...,10,Jon Spaihts,Scott Derrickson,Benedict Cumberbatch,Chiwetel Ejiofor,Rachel McAdams,While on a journey of physical and spiritual h...,"Marvel's ""Doctor Strange"" follows the story of...",7.498347,Action Adventure Fantasy
907,tt0383574,Pirates of the Caribbean: Dead Man's Chest,2006,07-07-2006,151,PG-13,PG-13,7.3,623360,1867,...,1,Ted Elliott,Terry Rossio,Johnny Depp,Orlando Bloom,Keira Knightley,Jack Sparrow races to recover the heart of Dav...,Once again we're plunged into the world of swo...,7.298865,Action Adventure Fantasy
906,tt0449088,Pirates of the Caribbean: At World's End,2007,05-25-2007,169,PG-13,PG-13,7.1,566300,1272,...,1,Ted Elliott,Terry Rossio,Johnny Depp,Geoffrey Rush,Orlando Bloom,"Captain Barbossa, Will Turner and Elizabeth Sw...","After Elizabeth, Will, and Captain Barbossa re...",7.099083,Action Adventure Fantasy
1512,tt0800369,Thor,2011,05-06-2011,115,PG-13,PG-13,7.0,702391,853,...,1,Ashley Miller,Zack Stentz,Chris Hemsworth,Natalie Portman,Tom Hiddleston,The powerful but arrogant god Thor is cast out...,The warrior Thor (Chris Hemsworth) is cast out...,6.999394,Action Adventure Fantasy
97,tt1477834,Aquaman,2018,12-21-2018,143,PG-13,PG-13,7.0,311374,3713,...,1,David Leslie Johnson-McGoldrick,Will Beall,Jason Momoa,Amber Heard,Willem Dafoe,"Arthur Curry (Jason Momoa), the human-born hei...","Arthur Curry (Jason Momoa), half human-half At...",6.998636,Action Adventure Fantasy
508,tt0411477,Hellboy II: The Golden Army,2008,07-11-2008,120,PG-13,PG-13,7.0,246472,369,...,1,Guillermo del Toro,Guillermo del Toro,Ron Perlman,Selma Blair,Doug Jones,The mythical world starts a rebellion against ...,After saving the world from the evil Russian m...,6.998279,Action Adventure Fantasy
1514,tt1981115,Thor: The Dark World,2013,11-08-2013,112,PG-13,PG-13,6.9,557492,649,...,1,Christopher L. Yost,Christopher Markus,Chris Hemsworth,Natalie Portman,Tom Hiddleston,When the Dark Elves attempt to plunge the univ...,"Thousands of years ago, a race of beings known...",6.899406,Action Adventure Fantasy
1602,tt0803096,Warcraft,2016,06-10-2016,123,PG-13,PG-13,6.8,230205,956,...,3,Charles Leavitt,Duncan Jones,Travis Fimmel,Paula Patton,Ben Foster,As an Orc horde invades the planet Azeroth usi...,When the world of the Orcs of Draenor is being...,6.798972,Action Adventure Fantasy
909,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,05-20-2011,136,PG-13,PG-13,6.6,455966,521,...,1,Ted Elliott,Terry Rossio,Johnny Depp,Pen lope Cruz,Geoffrey Rush,Jack Sparrow and Barbossa embark on a quest to...,Captain Jack Sparrow (Johnny Depp) crosses pat...,6.599892,Action Adventure Fantasy
